In [1]:
import numpy as np
import pandas as pd
import time
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from hyperopt import fmin, tpe, hp, Trials, STATUS_OK

In [2]:
# Загрузка данных из файла
data = pd.read_csv('diabetes.csv')

# Разделение признаков и целевой переменной
X = data.drop('Outcome', axis=1)
y = data['Outcome']

# Разделение на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [3]:
# Инициализация модели
rf = RandomForestClassifier(random_state=42)

# Гиперпараметры для случайного поиска
param_dist = {
    'n_estimators': np.arange(50, 201, 10),
    'max_depth': np.arange(5, 31, 5),
    'min_samples_split': np.arange(2, 11, 2),
    'min_samples_leaf': np.arange(1, 11, 2),
    'bootstrap': [True, False]
}

# Random Search
random_search = RandomizedSearchCV(
    estimator=rf,
    param_distributions=param_dist,
    n_iter=50,
    cv=5,
    verbose=1,
    random_state=42,
    n_jobs=-1
)

# Время и обучение
start_time = time.time()
random_search.fit(X_train, y_train)
end_time = time.time()

best_params_random = random_search.best_params_
best_score_random = random_search.best_score_
time_random = end_time - start_time

print("RandomSearch лучшие параметры:", best_params_random)
print("RandomSearch точность:", best_score_random)
print("Время RandomSearch:", time_random, "сек")


Fitting 5 folds for each of 50 candidates, totalling 250 fits
RandomSearch лучшие параметры: {'n_estimators': np.int64(60), 'min_samples_split': np.int64(4), 'min_samples_leaf': np.int64(7), 'max_depth': np.int64(20), 'bootstrap': False}
RandomSearch точность: 0.7899107023857124
Время RandomSearch: 51.39827370643616 сек


In [5]:
# Пространство поиска
space = {
    'n_estimators': hp.quniform('n_estimators', 50, 200, 10),
    'max_depth': hp.quniform('max_depth', 5, 30, 5),
    'min_samples_split': hp.quniform('min_samples_split', 2, 10, 2),
    'min_samples_leaf': hp.quniform('min_samples_leaf', 1, 10, 2),
    'bootstrap': hp.choice('bootstrap', [True, False])
}

# Целевая функция
def objective(params):
    params['n_estimators'] = int(params['n_estimators'])
    params['max_depth'] = int(params['max_depth'])
    params['min_samples_split'] = int(params['min_samples_split'])
    params['min_samples_leaf'] = int(params['min_samples_leaf'])

    clf = RandomForestClassifier(**params, random_state=42)
    clf.fit(X_train, y_train)
    preds = clf.predict(X_test)
    acc = accuracy_score(y_test, preds)
    return {'loss': -acc, 'status': STATUS_OK}

# Поиск через TPE
trials = Trials()
start_time = time.time()
best = fmin(
    fn=objective,
    space=space,
    algo=tpe.suggest,
    max_evals=50,
    trials=trials,
    rstate=np.random.default_rng(42)
)
end_time = time.time()

# Расшифровка
best['bootstrap'] = [True, False][best['bootstrap']]
best = {k: int(v) if k != 'bootstrap' else v for k, v in best.items()}

# Проверка точности
clf = RandomForestClassifier(**best, random_state=42)
clf.fit(X_train, y_train)
preds = clf.predict(X_test)
acc = accuracy_score(y_test, preds)
time_tpe = end_time - start_time

print("Hyperopt лучшие параметры:", best)
print("Hyperopt точность:", acc)
print("Время Hyperopt:", time_tpe, "сек")


100%|██████████| 50/50 [00:13<00:00,  3.81trial/s, best loss: -0.7727272727272727]
Hyperopt лучшие параметры: {'bootstrap': False, 'max_depth': 5, 'min_samples_leaf': 8, 'min_samples_split': 6, 'n_estimators': 100}
Hyperopt точность: 0.7727272727272727
Время Hyperopt: 13.25812554359436 сек


In [6]:
print("\n Сравнительная таблица:")
print(f"RandomizedSearchCV: Точность = {best_score_random:.4f}, Время = {time_random:.2f} сек")
print(f"Hyperopt TPE:        Точность = {acc:.4f}, Время = {time_tpe:.2f} сек")



 Сравнительная таблица:
RandomizedSearchCV: Точность = 0.7899, Время = 51.40 сек
Hyperopt TPE:        Точность = 0.7727, Время = 13.26 сек
